<a href="https://colab.research.google.com/github/Rayyan-Portfolio/Gen_Ai/blob/main/AI_teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages in Colab
!pip install fastapi uvicorn gtts nest_asyncio reportlab transformers torch

from fastapi import FastAPI, HTTPException, UploadFile, File
from gtts import gTTS
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel
from typing import Dict, List
import nest_asyncio
import uvicorn
import torch
from IPython.display import Audio, display
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

app = FastAPI()

# Load Whisper model for speech-to-text
asr_pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

# Load GPT-2 model for text generation
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Sample course outline from the document (Section A.2: Course Management)
COURSE_OUTLINE = {
    "course_name": "Biology Basics",
    "topics": [
        {
            "id": 1,
            "title": "Introduction to Biology",
            "description": "Overview of biology and its branches.",
            "duration": "15 minutes"
        },
        {
            "id": 2,
            "title": "Cell Structure",
            "description": "Basic structure and function of cells.",
            "duration": "20 minutes"
        },
        {
            "id": 3,
            "title": "Photosynthesis",
            "description": "Process of photosynthesis in plants.",
            "duration": "25 minutes"
        }
    ]
}

# Track progress
progress: Dict[str, int] = {"current_topic_id": 0}

# Function to generate content with GPT-2
def generate_content(prompt: str, max_length: int = 200):
    result = text_generator(prompt, max_length=max_length, num_return_sequences=1, truncation=True)[0]["generated_text"]
    # Clean up: Limit to roughly 200 words (~1000 characters)
    return result[:1000].strip()

@app.get("/course-outline")
async def get_course_outline():
    """Returns the course outline."""
    return {"course_name": COURSE_OUTLINE["course_name"], "topics": [{"id": t["id"], "title": t["title"]} for t in COURSE_OUTLINE["topics"]]}

@app.get("/start-lecture")
async def start_lecture():
    """Starts the lecture from the first topic."""
    progress["current_topic_id"] = 1
    topic = next((t for t in COURSE_OUTLINE["topics"] if t["id"] == 1), None)
    if not topic:
        raise HTTPException(status_code=404, detail="Course outline empty.")
    return await generate_lecture(topic)

@app.get("/next-lecture")
async def next_lecture():
    """Moves to the next topic."""
    current_id = progress["current_topic_id"]
    next_id = current_id + 1
    topic = next((t for t in COURSE_OUTLINE["topics"] if t["id"] == next_id), None)
    if not topic:
        raise HTTPException(status_code=404, detail="No more topics.")
    progress["current_topic_id"] = next_id
    return await generate_lecture(topic)

async def generate_lecture(topic: Dict):
    """Generates lecture content using GPT-2."""
    # Generate lecture text
    lecture_prompt = f"Generate a lecture on '{topic['title']}' focusing on: {topic['description']}"
    lecture_text = generate_content(lecture_prompt)

    # Generate whiteboard instructions
    whiteboard_prompt = f"Describe how to draw a simple diagram for '{topic['title']}' on a whiteboard."
    whiteboard_text = generate_content(whiteboard_prompt, max_length=100)

    # Generate example
    example_prompt = f"Provide a real-world example for '{topic['title']}'."
    example_text = generate_content(example_prompt, max_length=50)

    # Generate quiz
    quiz_prompt = f"Create a simple question and answer about '{topic['title']}'."
    quiz_text = generate_content(quiz_prompt, max_length=50)
    quiz = {"question": quiz_text.split("?")[0] + "?", "answer": quiz_text.split("?")[1].strip() if "?" in quiz_text else "Unknown"}

    # Generate audio
    tts = gTTS(text=lecture_text, lang="en")
    audio_file = f"{topic['title'].replace(' ', '_')}.mp3"
    tts.save(audio_file)

    # Generate handout (PDF)
    pdf_file = f"{topic['title'].replace(' ', '_')}.pdf"
    c = canvas.Canvas(pdf_file, pagesize=letter)
    c.drawString(100, 750, topic["title"])
    text_obj = c.beginText(100, 730)
    for line in lecture_text.split(". "):
        text_obj.textLine(line)
    c.drawText(text_obj)
    c.save()

    return {
        "topic": topic["title"],
        "text": lecture_text,
        "whiteboard": whiteboard_text,
        "example": example_text,
        "quiz": quiz,
        "audio": audio_file,
        "handout": pdf_file
    }

@app.get("/current-lecture")
async def current_lecture():
    """Returns the current lecture."""
    current_id = progress["current_topic_id"]
    if current_id == 0:
        raise HTTPException(status_code=400, detail="No lecture started.")
    topic = next((t for t in COURSE_OUTLINE["topics"] if t["id"] == current_id), None)
    if not topic:
        raise HTTPException(status_code=404, detail="Invalid topic.")
    return await generate_lecture(topic)

@app.post("/ask-question")
async def ask_question(audio: UploadFile = File(...)):
    """Handles voice questions using Whisper."""
    # Save uploaded audio file
    audio_file = "question.wav"
    with open(audio_file, "wb") as f:
        f.write(await audio.read())

    # Transcribe with Whisper
    transcription = asr_pipe(audio_file)["text"]

    # Simple rule-based response system (since GPT-2 isn’t great for Q&A dynamically)
    responses = {
        "what is biology": "Biology is the study of living organisms.",
        "what is a cell": "A cell is the basic unit of life.",
        "what is photosynthesis": "Photosynthesis is how plants make food using sunlight."
    }
    question = transcription.lower().strip()
    answer = responses.get(question, "I don’t have an answer for that yet. Try asking something like 'What is biology?'")

    # Generate audio response
    tts = gTTS(text=answer, lang="en")
    answer_audio = "answer.mp3"
    tts.save(answer_audio)

    return {"question": transcription, "answer": answer, "audio": answer_audio}

# Simulate API calls in Colab
async def simulate_lecture_call(endpoint: str):
    from fastapi.testclient import TestClient
    client = TestClient(app)
    response = client.get(endpoint)
    if response.status_code == 200:
        data = response.json()
        print(f"Topic: {data['topic']}")
        print(f"Lecture Text: {data['text']}")
        print(f"Whiteboard Instructions: {data['whiteboard']}")
        print(f"Example: {data['example']}")
        print(f"Quiz: {data['quiz']['question']} (Answer: {data['quiz']['answer']})")
        display(Audio(data['audio']))
        print(f"Handout saved as: {data['handout']}")
    else:
        print(f"Error: {response.json()}")

# Run server in background
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

import threading
server_thread = threading.Thread(target=run_server)
server_thread.start()

# Test the lecture
import asyncio
asyncio.run(simulate_lecture_call("/start-lecture"))
# Uncomment to test further
# asyncio.run(simulate_lecture_call("/next-lecture"))
# asyncio.run(simulate_lecture_call("/current-lecture"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 349.1/664.8 MB 26.8 MB/s eta 0:00:12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.8 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone m

/usr/local/lib/python3.11/dist-packages/torch/distributed/__init__.py:28: RuntimeWarning: coroutine 'Server.serve' was never awaited
  if is_available() and not torch._C._c10d_init():
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
ERROR:fastapi:Form data requires "python-multipart" to be installed. 
You can install "python-multipart" with: 

pip install python-multipart



RuntimeError: Form data requires "python-multipart" to be installed. 
You can install "python-multipart" with: 

pip install python-multipart


In [ ]:
!pip install gtts

In [ ]:
!pip install fastapi
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.7 MB/s eta 0:00:00
